In [1]:
import pandas as pd
import numpy as np
import datetime
import os
# Add the new folders of Jian later

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2)

write_folder="/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_"+str(last_saturday)+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

week_beginning=last_saturday-datetime.timedelta(days=28) # To be changed to 14

recent_3_weeks=[last_saturday-datetime.timedelta(days=x*7) for x in range(3)]

In [3]:
# Checking for the recent weeks that haven't moved to the folder
possible_recent_folders=["/home/jian/BigLots/MediaStorm_"+str(x)+"/" for x in recent_3_weeks]
recent_file_list=[]
for dirc in possible_recent_folders:
    list_recent=[x for x in list(recursive_file_gen(dirc)) if ("Master" in x) & (".txt" in x) ]
    recent_file_list=recent_file_list+list_recent
recent_file_df=pd.DataFrame({"path":recent_file_list,"date":[datetime.datetime.strptime(x.split("MasterWeekly")[1][:8],"%Y%m%d").date()-datetime.timedelta(days=3) for x in recent_file_list]},index=[x for x in range(len(recent_file_list))])


list_1_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/")) if ("Master" in x) & (".txt" in x) ]
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-2].split("_")[1],"%Y-%m-%d").date() for x in list_1_after_201806]
df_1_after_201806=pd.DataFrame({"date":folder_date,"path":list_1_after_201806},index=[x for x in range(len(list_1_after_201806))])
df_1_after_201806['date'].apply(lambda x: x.weekday()).unique()
df_1_after_201806=df_1_after_201806.sort_values("date").reset_index()
df_1_after_201806=df_1_after_201806[df_1_after_201806['date']>week_beginning]
del df_1_after_201806['index']

new_2_biweekly_files=df_1_after_201806.append(recent_file_df)

print(len(new_2_biweekly_files))
new_2_biweekly_files=new_2_biweekly_files['path'].unique().tolist()

4


In [4]:
new_2_biweekly_files

['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-02/MediaStormMasterWeekly20190305-112945-302.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-09/MediaStormMasterWeekly20190312-121512-232.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-16/MediaStormMasterWeekly20190319-112932-415.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-23/MediaStormMasterWeekly20190326-113052-887.txt']

In [5]:
if len(new_2_biweekly_files)==4: # Changed to weekly
    df_all_new_master = pd.DataFrame()
    for file in new_2_biweekly_files:
        df = pd.read_csv(file,nrows = None,sep= '|',usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'],
                         dtype =str)
        df_all_new_master = df_all_new_master.append(df,ignore_index = True)
    print(len(df_all_new_master.index))
    df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
    print(len(df_all_new_master.index))

342759
342759


In [6]:
# Remove previous week ids

previous_id_files_lists=list(recursive_file_gen("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update/crm_newscore_20190107/by_group_quadrant_24/"))

# previous_id_files_lists=[x for x in previous_id_files_lists if "Copy of 48 Stores_727.csv" not in x]
previous_id_files_lists=[x for x in previous_id_files_lists if ".csv" in x]
print(len(previous_id_files_lists))

72


In [7]:
import glob
uploaded_new_seg=list(recursive_file_gen("/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/"))
uploaded_new_seg=[x for x in uploaded_new_seg if ".csv" in x]

previous_id_files_lists=previous_id_files_lists+uploaded_new_seg

In [8]:
for file_uploaded in previous_id_files_lists:
    df = pd.read_csv(file_uploaded,usecols = ['email_address_hash'])
    previous_email_set=set(df['email_address_hash'].tolist())
    df_all_new_master = df_all_new_master[~df_all_new_master['email_address_hash'].isin(previous_email_set)]
    print(datetime.datetime.now(),len(df_all_new_master.index))

2019-03-27 10:56:28.206569 342753
2019-03-27 10:56:39.454252 342753
2019-03-27 10:56:40.206914 342752
2019-03-27 10:56:41.048260 342750
2019-03-27 10:56:42.213013 342748
2019-03-27 10:56:42.446286 342748
2019-03-27 10:56:42.601992 342748
2019-03-27 10:56:42.693928 342748
2019-03-27 10:56:42.769835 342748
2019-03-27 10:56:42.848099 342747
2019-03-27 10:56:42.968683 342747
2019-03-27 10:56:43.108031 342747
2019-03-27 10:56:43.190749 342747
2019-03-27 10:56:43.266714 342747
2019-03-27 10:56:43.347416 342747
2019-03-27 10:56:43.424837 342747
2019-03-27 10:56:43.557420 342747
2019-03-27 10:56:43.679815 342746
2019-03-27 10:56:43.748512 342746
2019-03-27 10:56:43.836801 342746
2019-03-27 10:56:43.905988 342746
2019-03-27 10:56:44.187705 342746
2019-03-27 10:56:44.658391 342745
2019-03-27 10:56:44.786489 342745
2019-03-27 10:56:44.892500 342745
2019-03-27 10:56:44.964748 342745
2019-03-27 10:56:56.117808 342743
2019-03-27 10:57:00.326963 342740
2019-03-27 10:57:08.792829 342737
2019-03-27 10:

In [9]:
df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
df_all_new_master = df_all_new_master.drop_duplicates('customer_id_hashed')
print(len(df_all_new_master.index))

342687


In [10]:
zipmap = pd.read_csv('/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/zip_with_ta_dma.csv',dtype = 'str')
zipmap['zipcodegroup'] = zipmap['revenue_flag']
zipmap = zipmap[['zip','zipcodegroup']].drop_duplicates('zip')
zipmap.columns = ['customer_zip_code','zipcodegroup']
df_all_new_master = pd.merge(df_all_new_master,zipmap,on ='customer_zip_code',how = 'left' )
print(df_all_new_master['zipcodegroup'].unique())
df_all_new_master['zipcodegroup'].fillna('T',inplace = True)
print(df_all_new_master['zipcodegroup'].unique())

last_saturday_str="_"+str(last_saturday).split("-")[1]+str(last_saturday).split("-")[2]+str(last_saturday).split("-")[0][2:]+"_"

for revenue_flag in ['P','S','T']:
    df = df_all_new_master[df_all_new_master['zipcodegroup']==revenue_flag]
    df = df[['customer_id_hashed','email_address_hash','customer_zip_code']]
    df['segment'] = 'NewReward' + last_saturday_str + revenue_flag
    print(revenue_flag,len(df.index))
    df.to_csv(write_folder+'NewReward_'+str(last_saturday)+'_'+revenue_flag+'.csv',index = False)
# To be uploaded to lr-big-lots uploads/big_lots_onboarding

['P' 'S' 'T' nan]
['P' 'S' 'T']
P 246635
S 63413
T 32639


In [11]:
df_all_new_master.groupby(['zipcodegroup'])['email_address_hash'].count().to_frame().reset_index()

,zipcodegroup,email_address_hash
0,P,246635
1,S,63413
2,T,32639
